# PCA MSC

In [ ]:
include("stuff.jl")

In [ ]:
cube_z_trans = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "z_trans")
cube_pca = load(joinpath(cubes_base_dir, "z_trans_pca.jld2"), "pca")
cube_pca_online = loadCube(joinpath(cubes_base_dir, "cube_pca_online"))
pca_no_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_no_msc_cube"))
pca_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_msc_cube"))
cube_extrema_msc = loadCube(joinpath(cubes_base_dir, "cube_extrema_msc"))
cube_extrema_no_msc = loadCube(joinpath(cubes_base_dir, "cube_extrema_no_msc"))

In [ ]:
n_seasons = 4
comp = 1
doy_idxs = (length(getAxis(MSCAxis, pca_msc_cube)) ÷ n_seasons) .* (0:n_seasons - 1) .+ 1 |> collect
ex_msc = [
    cube_extrema_msc[:, i] |> 
    x -> abs.(x) |> 
    x -> max(x...) |> 
    x -> (-x, x) 
    for i in 1:size(cube_extrema_msc, 2)
]
ex_no_msc = [
    cube_extrema_no_msc[:, i] |> 
    x -> abs.(x) |>
    x -> max(x...) |> 
    x -> (-x, x) 
    for i in 1:size(cube_extrema_no_msc, 2)
]

In [ ]:
R"""
pdf($(joinpath(fig_path, "msc_doy.pdf")), width = 12.05, height = 9)
data(coastsCoarse, package = "rworldmap")

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
          "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
          "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
              "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
              "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

r2a <- function(x) {
    x <- as.array(x)
    x <- x[1:600, image_x_min_col:1340, 1]
    x <- t(x)[,dim(x)[1]:1]
}


label_cex <- 1.5
scale_cex <- 1.3
image_x_min_col <- 250

doy_to_date <- function(doi) {
    format(as.Date(doi - 1, origin = "2016-01-01"), "%d %b")
}


prog <- function(x, max) {
    #x <- x - 1
    #pie(c(x / max, 1 - x / max), clockwise = TRUE, labels = NA, col = c("#000000", "#FFFFFF"))
    #plot(1:2, 1:2, type = "n", bty = "n", axes = FALSE)
    plot.new()
    text(0.5, 0.5, paste(doy_to_date((x - 1) * 8 + 1)), adj = c(0.5, 0.5), srt = 90, cex = scale_cex, xpd = NA)
}

clamp <- function(x, lims) {
    lo <- lims[1]
    hi <- lims[2]
    res <- x
    res[res < lo] <- lo
    res[res > hi] <- hi
    return(res)
}

rotr90 <- function(x) t(x[dim(x)[1]:1,])

asp_man <- 600 / (1340 - image_x_min_col)

clamp_fac <- 0.7
ex1 <- $([Float64.(ex_msc[1])...]) * clamp_fac
ex2 <- $([Float64.(ex_msc[2])...]) * clamp_fac
ex3 <- $([Float64.(ex_msc[3])...]) * clamp_fac

map1 <- clamp(t($(pca_msc_cube[ 1, 1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map2 <- clamp(t($(pca_msc_cube[12, 1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map3 <- clamp(t($(pca_msc_cube[23, 1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map4 <- clamp(t($(pca_msc_cube[35, 1, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)

map5 <- clamp(t($(pca_msc_cube[ 1, 2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map6 <- clamp(t($(pca_msc_cube[12, 2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map7 <- clamp(t($(pca_msc_cube[23, 2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map8 <- clamp(t($(pca_msc_cube[35, 2, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)

map9  <- clamp(t($(pca_msc_cube[ 1, 3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map10 <- clamp(t($(pca_msc_cube[12, 3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map11 <- clamp(t($(pca_msc_cube[23, 3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)
map12 <- clamp(t($(pca_msc_cube[35, 3, :, :] |> x -> convert(Array, x) |> missing_to_nan .|> Float64)), ex1)


layout(matrix(1:20, nrow = 5, ncol = 4), 
       heights = c(1, 1, 1, 1, 0.3), 
       widths = c(0.05, 1, 1, 1))
par(mar = c(0, 0, 0, 0))
prog(1, 46)
prog(12, 46)
prog(23, 46)
prog(35, 46)
plot.new()
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map1)[,100:720], zlim = ex1, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal1, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map2)[,100:720], zlim = ex1, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal1, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map3)[,100:720], zlim = ex1, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal1, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map4)[,100:720], zlim = ex1, abty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal1, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)

par(mar = c(4, 7, 0, 4))
image(matrix(1:100), col = pal1, axes = FALSE, bty = 'n', 
      xlab = expression(low~productivity %<-% PC[1] %->% high~productivity), 
      cex.lab = label_cex, useRaster = TRUE)
axis(1, 
     at = seq(    (ceiling(ex1[1]) - ex1[1]) / (ex1[2] - ex1[1]), 
              1 - (ex1[2] - floor(ex1[2]))   / (ex1[2] - ex1[1]), 
              length.out = 5),
     labels = seq(ceiling(ex1[1]), floor(ex1[2]), length.out = 5),
     cex.axis = scale_cex)

par(mar = c(0, 0, 0, 0))
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map5)[,100:720], zlim = ex2, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal2, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map6)[,100:720], zlim = ex2, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal2, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map7)[,100:720], zlim = ex2, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal2, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map8)[,100:720], zlim = ex2, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal2, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)

par(mar = c(4, 7, 0, 4))
image(matrix(1:100), col = pal2, axes = FALSE, bty = 'n', 
      xlab = expression(wet %<-% PC[2] %->% dry), 
      cex.lab = label_cex, useRaster = TRUE)
axis(1, 
     at = seq(    (ceiling(ex2[1]) - ex2[1]) / (ex2[2] - ex2[1]), 
              1 - (ex2[2] - floor(ex2[2]))   / (ex2[2] - ex2[1]), 
              length.out = 5),
     labels = seq(ceiling(ex2[1]), floor(ex2[2]), length.out = 5),
     cex.axis = scale_cex)

par(mar = c(0, 0, 0, 0))
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map9 )[,100:720], zlim = ex3, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal3, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map10)[,100:720], zlim = ex3, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal3, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map11)[,100:720], zlim = ex3, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal3, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-65, 90, by = 0.25), rotr90(map12)[,100:720], zlim = ex3, bty = 'n', axes = FALSE, xlab = NA, ylab = NA, col = pal3, useRaster = TRUE)
plot(coastsCoarse, add = TRUE)

par(mar = c(4, 7, 0, 4))
image(matrix(1:100), col = pal3, axes = FALSE, bty = 'n', 
      xlab = expression(dark %<-% PC[3] %->% light), 
      cex.lab = label_cex, useRaster = TRUE)
axis(1, 
     at = seq(    (ceiling(ex3[1]) - ex3[1]) / (ex3[2] - ex3[1]), 
              1 - (ex3[2] - floor(ex3[2]))   / (ex3[2] - ex3[1]), 
              length.out = 5),
     labels = seq(ceiling(ex3[1]), floor(ex3[2]), length.out = 5),
     cex.axis = scale_cex)

grid.text("(a)", 0,   1,                         just = c(-1, 1))
grid.text("(b)", 1/3, 1,                         just = c(-1, 1))
grid.text("(c)", 2/3, 1,                         just = c(-1, 1))

grid.text("(d)", 0,   3/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))
grid.text("(e)", 1/3, 3/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))
grid.text("(f)", 2/3, 3/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))

grid.text("(g)", 0,   2/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))
grid.text("(h)", 1/3, 2/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))
grid.text("(i)", 2/3, 2/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))


grid.text("(j)", 0,   1/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))
grid.text("(k)", 1/3, 1/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))
grid.text("(l)", 2/3, 1/4 * 4 / 4.3 + 0.3 / 4.3, just = c(-1, 1))


dev.off()
"""
show_pdf(
    "http://127.0.0.1:8333/files/results/fig/msc_doy.pdf",
    width = 900,
    height = 900
)

In [ ]:
rmprocs(workers())
addprocs(40)
@everywhere using WeightedOnlineStats

pca_cor_cube_sp = mapCube(
    pca_msc_cube,
    indims = InDims("MSC", "PCA"),
    outdims = OutDims(
        CategoricalAxis(:PcaAxis1, pca_axis.values), 
        CategoricalAxis(:PcaAxis2, pca_axis.values)
    )
) do xout, xin
    # using WeightedOnlineStats here because it ignores missing values.
    tmp = fit!(WeightedCovMatrix(), eachrow(xin), repeat([1.0], size(xin, 1))) |> cor
    if size(tmp) == (0, 0)
        xout[:] .= missing
    else
        xout .= tmp
    end
end

rmprocs(workers())

In [ ]:
rmprocs(workers())
addprocs(40)
@everywhere using WeightedOnlineStats

pca_cov_cube_sp = mapCube(
    pca_msc_cube,
    indims = InDims("MSC", "PCA"),
    outdims = OutDims(
        CategoricalAxis(:PcaAxis1, pca_axis.values), 
        CategoricalAxis(:PcaAxis2, pca_axis.values)
    )
) do xout, xin
    # using WeightedOnlineStats here because it ignores missing values.
    tmp = fit!(WeightedCovMatrix(), eachrow(xin), repeat([1.0], size(xin, 1))) |> cov
    if size(tmp) == (0, 0)
        xout[:] .= missing
    else
        xout .= tmp
    end
end

rmprocs(workers())

In [ ]:
plotMAP(pca_cor_cube_sp)

In [ ]:
plotMAP(pca_cov_cube_sp)

In [ ]:
pca_cor_cube_sp

In [ ]:
R"""
pdf($(joinpath(fig_path, "pca_msc_cor_sp.pdf")), width = 10, height = 15)
layout(matrix(c(1:3), ncol = 1))

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
          "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
          "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
              "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
              "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

cor12 <- $(pca_cor_cube_sp[1, 2, :,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])
cor13 <- $(pca_cor_cube_sp[1, 3, :,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])
cor23 <- $(pca_cor_cube_sp[2, 3, :,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])

crange12 <- c(-1, 1) # 0.8 * (max(abs(range(hyst12, na.rm = TRUE))) %>% { c(-., .) })
crange13 <- c(-1, 1) # 0.8 * (max(abs(range(hyst13, na.rm = TRUE))) %>% { c(-., .) })
crange23 <- c(-1, 1) # 0.8 * (max(abs(range(hyst23, na.rm = TRUE))) %>% { c(-., .) })

hist12 <- hist(cor12, plot = FALSE)
hist13 <- hist(cor13, plot = FALSE)
hist23 <- hist(cor23, plot = FALSE)

############ maps
par(mar = c(0, 0, 0, 0))
image(cor12[, 120:720], zlim = crange12, col = pal1, useRaster = TRUE, bty = 'n', axes = FALSE)
image(cor13[, 120:720], zlim = crange13, col = pal2, useRaster = TRUE, bty = 'n', axes = FALSE)
image(cor23[, 120:720], zlim = crange23, col = pal3, useRaster = TRUE, bty = 'n', axes = FALSE)

############ color bars and hists
par(fig = c(0.07, 0.10, (2 + 0.05) / 3, (2 + 0.5) / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange12[1], crange12[2], length.out = 100),
      t(seq(crange12[1], crange12[2], length.out = 100)), 
      col = pal1, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      #ylab = expression(atop("Area " * group("[", "std" ^ 2, "]"),
      #                       "clockwise"%<->% "counterclockwise")),
      ylab = "r",
      xlab = NA
)
rect(1, hist12$breaks[-length(hist12$breaks)], 1 + hist12$density, hist12$breaks[-1])


par(fig = c(0.07, 0.10, (1 + 0.05) / 3, (1 + 0.5) / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange13[1], crange13[2], length.out = 100),
      t(seq(crange13[1], crange13[2], length.out = 100)), 
      col = pal2, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      #ylab = expression(atop("Area " * group("[", "std" ^ 2, "]"),
      #                       "clockwise"%<->% "counterclockwise")),
      ylab = "r",
      xlab = NA
)
rect(1, hist13$breaks[-length(hist13$breaks)], 1 + hist13$density, hist13$breaks[-1])

par(fig = c(0.07, 0.10, 0.05 / 3, 0.5 / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange23[1], crange23[2], length.out = 100),
      t(seq(crange23[1], crange23[2], length.out = 100)), 
      col = pal3, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      #ylab = expression(atop("Area " * group("[", "std" ^ 2, "]"),
      #                       "clockwise"%<->% "counterclockwise")),
      ylab = "r",
      xlab = NA
)
rect(1, hist23$breaks[-length(hist23$breaks)], 1 + hist23$density, hist23$breaks[-1])

######## decoration
grid.text("(a)", 0,   1, just = c(0, 1))
grid.text("(b)", 0, 2/3, just = c(0, 1))
grid.text("(c)", 0, 1/3, just = c(0, 1))


dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/pca_msc_cor_sp.pdf", width = 700, height = 1000)

In [ ]:
R"""
pdf($(joinpath(fig_path, "pca_msc_cov_sp.pdf")), width = 20/3, height = 10)
data(coastsCoarse, package = "rworldmap")
layout(matrix(c(1:3), ncol = 1))

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
          "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
          "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
              "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
              "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

cov12 <- $(pca_cov_cube_sp[1, 2, :,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])
cov13 <- $(pca_cov_cube_sp[1, 3, :,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])
cov23 <- $(pca_cov_cube_sp[2, 3, :,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])

crange12 <- (max(abs(range(cov12, na.rm = TRUE))) %>% { c(-., .) })
crange13 <- (max(abs(range(cov13, na.rm = TRUE))) %>% { c(-., .) })
crange23 <- (max(abs(range(cov23, na.rm = TRUE))) %>% { c(-., .) })

hist12 <- hist(cov12, plot = FALSE)
hist13 <- hist(cov13, plot = FALSE)
hist23 <- hist(cov23, plot = FALSE)

############ maps
par(mar = c(0, 0, 0, 0))
image(seq(-180, 180, by = 0.25), seq(-60, 90, by = 0.25), cov12[, 120:720], zlim = crange12, col = pal1, useRaster = TRUE, bty = 'n', axes = FALSE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-60, 90, by = 0.25), cov13[, 120:720], zlim = crange13, col = pal2, useRaster = TRUE, bty = 'n', axes = FALSE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-60, 90, by = 0.25), cov23[, 120:720], zlim = crange23, col = pal3, useRaster = TRUE, bty = 'n', axes = FALSE)
plot(coastsCoarse, add = TRUE)

############ color bars and hists
par(fig = c(0.07, 0.10, (2 + 0.05) / 3, (2 + 0.5) / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange12[1], crange12[2], length.out = 100),
      t(seq(crange12[1], crange12[2], length.out = 100)), 
      col = pal1, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      ylab = expression("cov(" * PC[1] * "," * PC[2] * ")"),
      xlab = NA
)
rect(1, hist12$breaks[-length(hist12$breaks)], 1 + hist12$density, hist12$breaks[-1])


par(fig = c(0.07, 0.10, (1 + 0.05) / 3, (1 + 0.5) / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange13[1], crange13[2], length.out = 100),
      t(seq(crange13[1], crange13[2], length.out = 100)), 
      col = pal2, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      ylab = expression("cov(" * PC[1] * "," * PC[3] * ")"),
      xlab = NA
)
rect(1, hist13$breaks[-length(hist13$breaks)], 1 + hist13$density, hist13$breaks[-1])

par(fig = c(0.07, 0.10, 0.05 / 3, 0.5 / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange23[1], crange23[2], length.out = 100),
      t(seq(crange23[1], crange23[2], length.out = 100)), 
      col = pal3, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      ylab = expression("cov(" * PC[2] * "," * PC[3] * ")"),
      xlab = NA
)
rect(1, hist23$breaks[-length(hist23$breaks)], 1 + hist23$density, hist23$breaks[-1])

######## decoration
grid.text("(a)", 0,   1, just = c(0, 1))
grid.text("(b)", 0, 2/3, just = c(0, 1))
grid.text("(c)", 0, 1/3, just = c(0, 1))


dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/pca_msc_cov_sp.pdf", width = 600, height = 1000)